In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types


In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
import pandas as pd
pandas_df = pd.read_parquet('data/memphis_police_data.parquet')

In [ ]:
mpd_schema = types.StructType([
    types.StructField("crime_id", types.StringType(), True),
    types.StructField("offense_date", types.StringType(), True),
    types.StructField("agency_crimetype_id", types.StringType(), True),
    types.StructField("city", types.StringType(), True),
    types.StructField("state", types.StringType(), True),
    types.StructField("masked_address", types.StringType(), True),
    types.StructField("category", types.StringType(), True),
    types.StructField("coord1", types.DoubleType(), True),
    types.StructField("coord2", types.DoubleType(), True),
    #types.StructField("location", types.MapType(types.StringType(), types.StringType()), True),
    types.StructField("offense_date_datetime", types.TimestampType(), True),
])

In [ ]:
filepath = 'data/memphis_police_data.parquet'

In [ ]:
df = spark.read.schema(mpd_schema).parquet(filepath)

In [ ]:
df.show()

In [ ]:
from pyspark.sql.functions import to_date
df = df.withColumn('offense_day', to_date('offense_date_datetime'))

In [ ]:
df.registerTempTable('memphis')

In [ ]:
pandas_df.columns

In [ ]:
results_df = spark.sql('''
SELECT agency_crimetype_id,
offense_day,
FIRST(category),
COUNT(agency_crimetype_id) AS daily_crimes
FROM memphis
GROUP BY 1, 2
SORT BY daily_crimes DESC
''')
results_df.show()

In [ ]:
# rename columns for report

In [ ]:
# write report df to parquet